In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
df_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
print("Data train shape = ", df_train.shape)
df_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
print("Data test shape = ", df_test.shape)

In [ ]:
df_train.head()

In [ ]:
df_test.tail()

In [ ]:
x_train = df_train.iloc[:, 1:].values.astype('float32')
y_train = df_train.iloc[:, 0].values.astype('int32')

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_train.shape

In [ ]:
y_train = to_categorical(y_train)
num_classes = y_train.shape[1]
num_classes

In [ ]:
x_dtrain, x_dvalidation, y_dtrain, y_dvalidation = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print("x_dtrain shape = ", x_dtrain.shape)
print("x_dvalidation shape = ", x_dvalidation.shape)
print("y_dtrain shape = ", y_dtrain.shape)
print("y_dvalidation shape = ", y_dvalidation.shape)

In [ ]:
def make_model():
    model = Sequential()
    model.add(Conv2D(16, (3, 3), input_shape=(28, 28, 1), padding='same', activation='relu'))
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    return model

In [ ]:
model = make_model()
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_dtrain, y_dtrain, batch_size=128, epochs=100)

In [ ]:
diagram = pd.DataFrame(history.history)
diagram.loc[:, ['accuracy', 'loss']].plot()

In [ ]:
df_submission = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
df_submission

In [ ]:
testing = df_test.iloc[:, :].values.astype('float32')
testing = testing.reshape(testing.shape[0], 28, 28, 1)
testing.shape

In [ ]:
y_pred = model.predict(testing)
y_pred = np.argmax(y_pred, axis=1)
submission = pd.DataFrame({'ImageId': np.arange(1, len(y_pred)+1), 'Label': y_pred})
submission.to_csv('/kaggle/working/submission.csv', index=False, header=True)